In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import logging
from pkg import u, m

clen_values, photon_energy_values = [1.5, 2.5, 3.5], [6000, 7000, 8000]
param_matrix = u.parameter_matrix(clen_values, photon_energy_values)

# parameters
dataset = '01'
threshold = 1
clen = 1.5 # meters 
photon_energy = 6000 # eV/ 6 keV

# instances
pm = u.PathManager()
peak_paths, water_peak_paths, labels, water_background_path = pm.select_dataset('01')
p = u.Processor(paths=pm, dataset=dataset)
dm = u.DatasetManager(paths=pm, dataset=dataset, transform=None)


true parameter matrix...

Found water background image: water01.h5
Number of peak images: 10
Number of water images: 10
Number of label images: 10
Check: Path to water background image: /Users/adamkurth/Documents/vscode/CXFEL/cxls_hitfinder/images/water/01/water01.h5



In [2]:
# generate peak/overlay/label and upate attributes
# overlaying the correct water01.h5 path to the peak images
# p.process_directory(dataset=dataset, clen=clen, photon_energy=photon_energy)

# peak, label, overlay, background are valid types
u.check_attributes(paths=pm, dataset=dataset, type='peak') 

train_loader, test_loader = u.prepare(data_manager=dm, batch_size=10)


true parameter matrix...

All files in dataset 01 of type 'peak' have matching attributes.

Data prepared.
Train size: 8
Test size: 2
Batch size: 10
Number of batches in train_loader: 1 



In [3]:
# models
models = [
    m.BasicCNN1(),
    m.BasicCNN2(),
    m.ResNet50BraggPeakClassifier(),
    m.DenseNetBraggPeakClassifier()
]

base1 = m.BasicCNN1(input_channels=1, output_channels=1, heatmap_size=(2163, 2069))
base2 = m.BasicCNN2(input_channels=1, output_channels=1, heatmap_size=(2163, 2069))
res50 = m.ResNet50BraggPeakClassifier(input_channels=1, output_channels=1, heatmap_size=(2163, 2069))
dense = m.DenseNetBraggPeakClassifier(input_channels=1, output_channels=1, heatmap_size=(2163, 2069))

Running `base1` model

In [6]:
device = u.get_device()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(base1.parameters(), lr=0.001)

cfg = {
    'model': base1,
    'device': device,
    'train_loader': train_loader,
    'test_loader': test_loader,
    'num_epochs': 5,
    'batch_size': 10,
    'num_classes': 1,
    'lr': 0.001
    }

t = u.TrainTestModels(model=base1, loader=[train_loader, test_loader], criterion=criterion, optimizer=optimizer, device=device, cfg=cfg)
t.train_model()

Model testing: BasicCNN1
-- epoch 0
Test accuracy: 0.0011854093754664063
-- epoch 1
Test accuracy: 0.0011854093754664063
-- epoch 2
Test accuracy: 0.0011854093754664063
-- epoch 3
Test accuracy: 0.0011854093754664063
-- epoch 4
Test accuracy: 0.0011854093754664063
